# Train and Generate Blur Kernels

This code trains a GAN based on WGAN architecture to generate synthetic blur kernels. The GAN is trained on kerenel approximated from low resolution images using Dark Channels.

Code by by Ruofan Zhou, Sabine Süsstrunk with slight modifications for userbility.

View the original Github Repo: 
[Kernel Modeling Super-Resolution on Real Low-Resolution Images](https://github.com/IVRL/Kernel-Modeling-Super-Resolution)

## Train Model

In [ ]:
from utils.wgan_clipping import WGAN_CP
from utils.data_loader import FolderDataset
from torch.utils.data import DataLoader


def train():
    # Modify MODEL_PATH to point to the directory to store the trained model
    # IMG_OUT and INT_IMG point to directors to store images as the training progresses
    save_path = {
    'MODEL_PATH' : 'drive/My Drive/Models/',
    'IMG_OUT' : 'drive/My Drive/ProgressImages/',
    'INT_IMG' : 'drive/My Drive/IntImages/'
    }

    model = WGAN_CP(save_path,channels=1, generator_iters=400000)
    dataset = 'drive/My Drive/Matlab/'
    # modify the following line for the dataset folder
    train_set = FolderDataset(dataset=dataset)
    train_loader = DataLoader(dataset=train_set, num_workers=1, batch_size=128, shuffle=True)
    
    model.train(train_loader)


if __name__ == '__main__':
    train()

## Load Model and Generate Kernels


In [ ]:
from utils.wgan_clipping import WGAN_CP
import torch
import torch.nn as nn
from torch.autograd import Variable
import time as t
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import os
from utils.tensorboard_logger import Logger
from torchvision import utils
import numpy as np
import cv2
from scipy.io import savemat

def normalize_kernel(k):
    k = k.numpy()
    k = np.clip(k, 0, 1)
    k = k/sum(sum(k))
    return k  

# modify the following line to the folder of output
outputdir = 'drive/My Drive/generated_kernels/'
# modify the following line to the number of kernels that needs to be generated
num_generate = 800

if not os.path.exists(outputdir):
    os.mkdir(outputdir)

# loading model
save_path = {
    'MODEL_PATH' : 'drive/My Drive/Models/',
    'IMG_OUT' : 'drive/My Drive/ProgressImages/',
    'INT_IMG' : 'drive/My Drive/IntImages/'
    }
model = WGAN_CP(save_path)
D_model_path = 'discriminator.pkl'
G_model_path = 'generator.pkl'
model.load_model(D_model_path, G_model_path)


z = Variable(torch.randn(num_generate, 100, 1, 1)).cuda()
samples = model.G(z)
samples = samples.data.cpu()
for i in range(num_generate):
    kernel = normalize_kernel(samples[i, 0])
    savemat('%s%d.mat'%(outputdir,i), {'kernel':kernel})
    if i%100 == 0:
      print('Completed {}'.format(i))

        # for plotting
    #kernel = kernel/np.max(kernel)
    #cv2.imwrite('%s%d.png'%(outputdir,i), kernel*255)